In [1]:
import pandas as pd
import re

import past_guest_module as pm

import sys
sys.path.append('../zoho_data_app/')

import main_module as m

%load_ext autoreload
%autoreload 2

In [2]:

path = r'C:\Users\fajar.fatoni\Documents\Python\Data\check_country_code_analytic.xlsx'
df_analytic = pd.read_excel(path, dtype='string', sheet_name='Sheet1')
count_data_awal = df_analytic.shape[0]
df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KB052562,Karma Resorts,Other,Past Guests,Karma Bavaria,PF325162,Mrs,Diane,Glover,Female,...,2025-08-19 00:00:00,8,WYNDHAM,WYNDHAM,WYNDHAM,3,1,0,Email and Phone,NO
1,KK067294,Karma Resorts,Other,Past Guests,Karma Kandara,PF298590,Mrs,Fui,Brown,Female,...,2025-08-20 00:00:00,4,DIGITAL,GHPA,DIGITAL,2,0,0,Email and Phone,NO
2,KK074932,Karma Resorts,Other,Past Guests,Karma Kandara,PF315763,Mr.,John Anderson Norwood,Moffat,Male,...,2025-08-21 00:00:00,5,HOUSE_USE,V_OWNER_NRB,COMP,2,2,0,Email and Phone,NO
3,KK069910,Karma Resorts,Other,Past Guests,Karma Kandara,PF303975,Ms,Carlene,Bourke,Unknown,...,2025-08-25 00:00:00,3,DIGITAL,GHPA,DIGITAL,2,0,0,Email and Phone,NO
4,KK078517,Karma Resorts,Other,Past Guests,Karma Kandara,PF325408,Mrs,Elena,Dzhashiashvili,Female,...,2025-08-22 00:00:00,14,HOUSE_USE,V_OWNER_NRB,COMP,4,2,0,Email and Phone,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102,KSM015292,Karma Resorts,Other,Past Guests,Karma St Martins,PF310868,<NA>,Lyndon,Clark,Unknown,...,2025-08-18 00:00:00,3,OTA,OTA,RETAIL,2,0,0,Phone Only,NO
103,KSM015136,Karma Resorts,Other,Past Guests,Karma St Martins,PF309285,<NA>,Candice,Cook,Unknown,...,2025-08-18 00:00:00,4,OTA,OTA,RETAIL,2,0,0,Phone Only,NO
104,KK073838,Karma Resorts,Other,Past Guests,Karma Kandara,PF312825,Mr,Daniel,Whitby,Male,...,2025-08-18 00:00:00,14,OTA,OTA,RETAIL,2,0,0,Phone Only,NO
105,KB052564,Karma Resorts,Other,Past Guests,Karma Bavaria,PF325166,Mrs,Chin Cheak,Yu,Female,...,2025-08-21 00:00:00,2,WYNDHAM,WYNDHAM,WYNDHAM,4,0,0,Email and Phone,NO


In [3]:
df_analytic.columns

Index(['Booking_Id', 'Lead Brand', 'Lead Sub- Brand', 'Lead Source',
       'Resort_Name', 'ProfileRef', 'Salutation', 'First_Name', 'Surname',
       'Gender', 'Language', 'DOB', 'Street', 'Area', 'Town', 'County',
       'Country', 'Nationality', 'PostCode', 'TelNo', 'MobileNo', 'Email',
       'Exclude From Mailing', 'Opt In', 'Booking Status', 'DateArrival',
       'DateDepart', 'No. of Nights', 'Market_Segment', 'Media_Code',
       'Media Code Group', 'No. of Adult', 'No. of Children', 'No. of Infants',
       'Contact Type', 'Odyssey Members'],
      dtype='object')

In [4]:
df_analytic.fillna('empty', inplace=True)

In [5]:
path = r'G:\.shortcut-targets-by-id\0B_d3smBjzo04UUNBME5GOVhoMjA\_ODYSSEY\Phone Members.xlsx'
df_phone = pd.read_excel(path, dtype='string', sheet_name='Sheet1')

path = r'G:\.shortcut-targets-by-id\0B_d3smBjzo04UUNBME5GOVhoMjA\_ODYSSEY\Email Members.xlsx'
df_email = pd.read_excel(path, dtype='string', sheet_name='Sheet1')


In [6]:
m.clean_space(df_email, 'Email')
df_email['Email'] = m.lowercase(df_email, 'Email')

c:\Users\fajar.fatoni\Documents\Python\Task Offline\past_guest_toni_app\../zoho_data_app\main_module.py:127: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[df_columns].replace(' ','', regex=True, inplace=True)


In [7]:
df_analytic.loc[
    (df_analytic['TelNo'] == 'empty') & (df_analytic['MobileNo'] != 'empty'), 
    'TelNo'] = df_analytic['MobileNo']
df_analytic.loc[df_analytic['TelNo'] == df_analytic['MobileNo'],'MobileNo'] = 'empty'

In [8]:
# hitung jumlah duplicate
count_duplicate_general = df_analytic.loc[df_analytic.duplicated(subset=['TelNo','MobileNo','Email'])].shape[0]

df_analytic.loc[df_analytic.duplicated(subset=['TelNo','MobileNo','Email'])]

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
26,KB051544,Karma Resorts,Other,Past Guests,Karma Bavaria,PF319820,Mr,Hameed,Sayed,Male,...,2025-08-22 00:00:00,1,COMP,GER_OWE,COMP,4,1,0,Email Only,NO
32,KB052796,Karma Resorts,Other,Past Guests,Karma Bavaria,PF071015,empty,Karin,Brahim,Female,...,2025-08-18 10:30:00,0,LGR,LGR,GROUP,1,0,0,Email and Phone,NO
43,KK067137,Karma Resorts,Other,Past Guests,Karma Kandara,PF295577,empty,Sourab,Pramanik,Unknown,...,2025-08-22 00:00:00,2,COMP,RESERVED,KC,1,0,0,Email and Phone,NO
44,KK067115,Karma Resorts,Other,Past Guests,Karma Kandara,PF295577,empty,Sourab,Pramanik,Unknown,...,2025-08-22 00:00:00,2,COMP,RESERVED,KC,1,0,0,Email and Phone,NO
45,KK067124,Karma Resorts,Other,Past Guests,Karma Kandara,PF295577,empty,Sourab,Pramanik,Unknown,...,2025-08-23 00:00:00,2,COMP,RESERVED,KC,1,0,0,Email and Phone,NO
46,KK067136,Karma Resorts,Other,Past Guests,Karma Kandara,PF295577,empty,Sourab,Pramanik,Unknown,...,2025-08-23 00:00:00,2,COMP,RESERVED,KC,1,0,0,Email and Phone,NO
69,KSM016079,Karma Resorts,Other,Past Guests,Karma St Martins,PF067424,Mr,Karma St. Martin'S,Management,Male,...,2025-08-19 00:00:00,8,COMP,TEL,RETAIL,1,0,0,Email and Phone,NO
79,KSM015816,Karma Resorts,Other,Past Guests,Karma St Martins,PF316875,Mrs,Mihaela,Reade,Unknown,...,2025-08-18 00:00:00,3,DIR,TEL,RETAIL,2,0,0,Email and Phone,NO
84,KSM016719,Karma Resorts,Other,Past Guests,Karma St Martins,PF325933,Mrs,Sarah,Hyatt-Williams,Unknown,...,2025-08-21 00:00:00,1,DIR,TEL,RETAIL,2,0,0,Email and Phone,NO


In [9]:
df_analytic.drop_duplicates(subset=['TelNo','MobileNo','Email'], inplace=True)
df_analytic.reset_index(drop=True, inplace=True)

# hitung jumlah setelah value duplicate dihapus
count_after_dedup_duplicate = df_analytic.shape[0]

df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KB052562,Karma Resorts,Other,Past Guests,Karma Bavaria,PF325162,Mrs,Diane,Glover,Female,...,2025-08-19 00:00:00,8,WYNDHAM,WYNDHAM,WYNDHAM,3,1,0,Email and Phone,NO
1,KK067294,Karma Resorts,Other,Past Guests,Karma Kandara,PF298590,Mrs,Fui,Brown,Female,...,2025-08-20 00:00:00,4,DIGITAL,GHPA,DIGITAL,2,0,0,Email and Phone,NO
2,KK074932,Karma Resorts,Other,Past Guests,Karma Kandara,PF315763,Mr.,John Anderson Norwood,Moffat,Male,...,2025-08-21 00:00:00,5,HOUSE_USE,V_OWNER_NRB,COMP,2,2,0,Email and Phone,NO
3,KK069910,Karma Resorts,Other,Past Guests,Karma Kandara,PF303975,Ms,Carlene,Bourke,Unknown,...,2025-08-25 00:00:00,3,DIGITAL,GHPA,DIGITAL,2,0,0,Email and Phone,NO
4,KK078517,Karma Resorts,Other,Past Guests,Karma Kandara,PF325408,Mrs,Elena,Dzhashiashvili,Female,...,2025-08-22 00:00:00,14,HOUSE_USE,V_OWNER_NRB,COMP,4,2,0,Email and Phone,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,KSM015292,Karma Resorts,Other,Past Guests,Karma St Martins,PF310868,empty,Lyndon,Clark,Unknown,...,2025-08-18 00:00:00,3,OTA,OTA,RETAIL,2,0,0,Phone Only,NO
94,KSM015136,Karma Resorts,Other,Past Guests,Karma St Martins,PF309285,empty,Candice,Cook,Unknown,...,2025-08-18 00:00:00,4,OTA,OTA,RETAIL,2,0,0,Phone Only,NO
95,KK073838,Karma Resorts,Other,Past Guests,Karma Kandara,PF312825,Mr,Daniel,Whitby,Male,...,2025-08-18 00:00:00,14,OTA,OTA,RETAIL,2,0,0,Phone Only,NO
96,KB052564,Karma Resorts,Other,Past Guests,Karma Bavaria,PF325166,Mrs,Chin Cheak,Yu,Female,...,2025-08-21 00:00:00,2,WYNDHAM,WYNDHAM,WYNDHAM,4,0,0,Email and Phone,NO


In [10]:
find_member_booking = df_analytic.copy()

find_member_booking.loc[find_member_booking['TelNo'].isin(df_phone['Mobile']), 'Same Value Find'] = 'Member-Mobile' 
find_member_booking.loc[find_member_booking['MobileNo'].isin(df_phone['Mobile']), 'Same Value Find'] = 'Member-Mobile'

find_member_booking.loc[find_member_booking['Email'].isin(df_email['Email']), 'Same Value Find'] = 'Member-Email'
find_member_booking.drop(find_member_booking.loc[find_member_booking['Same Value Find'].isna()].index, inplace=True)

find_member_booking.replace('empty', '', inplace=True)

In [11]:
find_member_booking.to_csv(r'C:\Users\fajar.fatoni\Documents\Python\Data\cek_member_zoho_analytic.csv', index=False)

In [12]:
count_dedup_member = df_analytic.loc[
    
    df_analytic['TelNo'].isin(df_phone['Mobile']) | 
    df_analytic['MobileNo'].isin(df_phone['Mobile']) |

    df_analytic['Email'].isin(df_email['Email'])].shape[0]


df_analytic.loc[
    
    df_analytic['TelNo'].isin(df_phone['Mobile']) | 
    df_analytic['MobileNo'].isin(df_phone['Mobile']) |

    df_analytic['Email'].isin(df_email['Email'])]


,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
3,KK069910,Karma Resorts,Other,Past Guests,Karma Kandara,PF303975,Ms,Carlene,Bourke,Unknown,...,2025-08-25 00:00:00,3,DIGITAL,GHPA,DIGITAL,2,0,0,Email and Phone,NO
18,KK078342,Karma Resorts,Other,Past Guests,Karma Kandara,PF077477,Mr,Jesper,Solgaard,Male,...,2025-08-21 00:00:00,11,HOUSE_USE,V_OWNER,COMP,4,0,0,Email and Phone,NO
27,KB052282,Karma Resorts,Other,Past Guests,Karma Bavaria,PF304831,Mrs,Alexandra,Babic,Female,...,2025-08-19 00:00:00,1,COMP,GER_OWE,COMP,2,0,0,Email and Phone,NO
28,KB050654,Karma Resorts,Other,Past Guests,Karma Bavaria,PF061147,Frau,Valentyna,Gernert,Female,...,2025-08-24 00:00:00,7,COMP,GER_OWE,COMP,2,0,0,Email and Phone,NO
30,KB052787,Karma Resorts,Other,Past Guests,Karma Bavaria,PF071015,empty,Karin,Brahim,Female,...,2025-08-18 10:30:00,0,LGR,LGR,GROUP,1,0,0,Email and Phone,NO
34,KK068177,Karma Resorts,Other,Past Guests,Karma Kandara,PF246401,Mr.,Torsten,Kiedel,Male,...,2025-08-18 00:00:00,11,HOUSE_USE,GER_OWE,COMP,2,3,0,Email and Phone,NO
40,KK067127,Karma Resorts,Other,Past Guests,Karma Kandara,PF295577,empty,Sourab,Pramanik,Unknown,...,2025-08-22 00:00:00,1,COMP,RESERVED,KC,1,0,0,Email and Phone,NO
62,KSM015378,Karma Resorts,Other,Past Guests,Karma St Martins,PF067424,Mr,Karma St. Martin'S,Management,Male,...,2025-08-22 00:00:00,3,COMP,STAFF,COMP,1,0,0,Email and Phone,NO
65,KSM016674,Karma Resorts,Other,Past Guests,Karma St Martins,PF283587,Mrs,Jessica,Kelly,Unknown,...,2025-08-19 00:00:00,7,DIR,WALKIN,RETAIL,2,0,0,Email and Phone,NO
87,KSM016571,Karma Resorts,Other,Past Guests,Karma St Martins,PF324447,Mr,Davey And,Gilbert,Unknown,...,2025-08-19 00:00:00,1,COMP,STAFF,COMP,2,0,0,Phone Only,NO


In [13]:
df_analytic.drop(df_analytic.loc[
    
    df_analytic['TelNo'].isin(df_phone['Mobile']) | 
    df_analytic['MobileNo'].isin(df_phone['Mobile']) |

    df_analytic['Email'].isin(df_email['Email'])
].index, inplace=True)

df_analytic.reset_index(drop=True, inplace=True)

count_after_dedup_member = df_analytic.shape[0]

In [14]:
count_data_akhir = df_analytic.shape[0]

In [15]:
df_analytic.sort_values('DateDepart', ascending=True, inplace=True)
df_analytic.reset_index(drop=True, inplace=True)
df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KB047546,Karma Resorts,Other,Past Guests,Karma Bavaria,PF301542,empty,Jan,Stifter,Unknown,...,2025-08-18 00:00:00,3,OTA,OTA,RETAIL,4,0,0,Phone Only,NO
1,KSM015815,Karma Resorts,Other,Past Guests,Karma St Martins,PF316875,Mrs,Mihaela,Reade,Unknown,...,2025-08-18 00:00:00,3,DIR,TEL,RETAIL,2,0,0,Email and Phone,NO
2,KB050840,Karma Resorts,Other,Past Guests,Karma Bavaria,PF315602,Mr,Shachar,Berlinsky,Male,...,2025-08-18 00:00:00,1,OTA,OTA,RETAIL,5,0,0,Phone Only,NO
3,KSM014744,Karma Resorts,Other,Past Guests,Karma St Martins,PF303357,empty,Max,Doppelbauer,Unknown,...,2025-08-18 00:00:00,2,OTA,OTA,RETAIL,2,0,0,Phone Only,NO
4,KSM014694,Karma Resorts,Other,Past Guests,Karma St Martins,PF302099,Mr,Graham,Willoughby,Unknown,...,2025-08-18 00:00:00,7,DIR,GFBL,DIGITAL,2,0,0,Email and Phone,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,KSM016799,Karma Resorts,Other,Past Guests,Karma St Martins,PF326594,Mr,Sahil,Khanna,Unknown,...,2025-08-24 00:00:00,3,DIR,TEL,RETAIL,2,1,0,Phone Only,NO
84,KK077785,Karma Resorts,Other,Past Guests,Karma Kandara,PF323494,empty,Tully,Jay,Unknown,...,2025-08-24 00:00:00,4,OTA,OTA,RETAIL,2,0,0,Phone Only,NO
85,KB046763,Karma Resorts,Other,Past Guests,Karma Bavaria,PF297268,Dr.,Johannes,Gansmeier,Unknown,...,2025-08-24 00:00:00,1,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
86,KB052029,Karma Resorts,Other,Past Guests,Karma Bavaria,PF322306,Frau,Tanya,Armbruster,Female,...,2025-08-24 00:00:00,3,OTA,OTA,RETAIL,4,0,0,Email and Phone,NO


In [16]:
df_analytic['DOB'].loc[df_analytic['DOB']=='']

Series([], Name: DOB, dtype: string)

In [17]:
df_analytic['DOB'].replace('empty', '', inplace=True)
dob_con_date = pd.to_datetime(df_analytic['DOB'], errors='coerce')
dob_date = dob_con_date.dt.strftime('%Y-%m-%d')

arr_con_date = pd.to_datetime(df_analytic['DateArrival'])
arr_date = arr_con_date.dt.strftime('%Y-%m-%d')

dep_con_date = pd.to_datetime(df_analytic['DateDepart'])
dep_date = dep_con_date.dt.strftime('%Y-%m-%d')

df_analytic['DOB'] = dob_date
df_analytic['DateArrival'] = arr_date
df_analytic['DateDepart'] = dep_date


C:\Users\fajar.fatoni\AppData\Local\Temp\ipykernel_22768\1513714307.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_analytic['DOB'].replace('empty', '', inplace=True)


In [18]:
df_analytic.replace('empty', '', inplace=True)

df_analytic.sort_values('DateDepart', ascending=True, inplace=True)
df_analytic.reset_index(drop=True, inplace=True)
df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KB047546,Karma Resorts,Other,Past Guests,Karma Bavaria,PF301542,,Jan,Stifter,Unknown,...,2025-08-18,3,OTA,OTA,RETAIL,4,0,0,Phone Only,NO
1,KSM016306,Karma Resorts,Other,Past Guests,Karma St Martins,PF321268,Mrs,Niki,Parker,Unknown,...,2025-08-18,3,DIR,GHPA,DIGITAL,2,0,0,Email and Phone,NO
2,KK075402,Karma Resorts,Other,Past Guests,Karma Kandara,PF317028,Ms,Sarah,Stavrow,Unknown,...,2025-08-18,3,OTA,OTA,RETAIL,2,0,0,Phone Only,NO
3,KK073838,Karma Resorts,Other,Past Guests,Karma Kandara,PF312825,Mr,Daniel,Whitby,Male,...,2025-08-18,14,OTA,OTA,RETAIL,2,0,0,Phone Only,NO
4,KSM015136,Karma Resorts,Other,Past Guests,Karma St Martins,PF309285,,Candice,Cook,Unknown,...,2025-08-18,4,OTA,OTA,RETAIL,2,0,0,Phone Only,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,KSM016811,Karma Resorts,Other,Past Guests,Karma St Martins,PF326715,Mr,Stacy,Flick,Unknown,...,2025-08-24,5,DIR,TEL,RETAIL,2,2,1,Phone Only,NO
84,KSM016799,Karma Resorts,Other,Past Guests,Karma St Martins,PF326594,Mr,Sahil,Khanna,Unknown,...,2025-08-24,3,DIR,TEL,RETAIL,2,1,0,Phone Only,NO
85,KK077785,Karma Resorts,Other,Past Guests,Karma Kandara,PF323494,,Tully,Jay,Unknown,...,2025-08-24,4,OTA,OTA,RETAIL,2,0,0,Phone Only,NO
86,KB046763,Karma Resorts,Other,Past Guests,Karma Bavaria,PF297268,Dr.,Johannes,Gansmeier,Unknown,...,2025-08-24,1,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO


In [19]:
print("==========================================================")
print("Summary")
print("==========================================================")
print("Jumlah Data Setelah Script Past Guest Sebelumnya : "+str(count_data_awal))
print("Jumlah Data Yang Duplicate : "+str(count_duplicate_general))
print("Jumlah Data Setelah Hapus Duplicate : "+str(count_after_dedup_duplicate))
print("Jumlah Data Yang Termasuk Member: "+str(count_dedup_member))
print("Jumlah Data Setelah Hapus Member : "+str(count_after_dedup_member))
print("Jumlah Data Akhir : "+str(count_data_akhir))

Summary
Jumlah Data Setelah Script Past Guest Sebelumnya : 107
Jumlah Data Yang Duplicate : 9
Jumlah Data Setelah Hapus Duplicate : 98
Jumlah Data Yang Termasuk Member: 10
Jumlah Data Setelah Hapus Member : 88
Jumlah Data Akhir : 88


In [20]:
df_analytic.to_excel(r'C:\Users\fajar.fatoni\Documents\Python\Data\final_analytics.xlsx', index=False)